In [1]:
from pntk.elements import *
from pntk.petri_net import *
from pntk.example_nets import *
import numpy as np

In [ ]:
efm_net = EmptyNet('efm_net')
initial_file_path = 'initial_file/test_net_2.csv'
efm_net.init_by_csv(initial_file_path)